# Dilithium - Glitches

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F4'
SS_VER = 'SS_VER_2_1'

TIMEOUT = 510
N = 256
NUM_ITERS = 64
BETA = 78

In [2]:
import logging
logging.basicConfig(level=logging.NOTSET)
logging.getLogger('io.github.alex1s.python-dilithium').setLevel(logging.WARNING)
logging.getLogger().setLevel(logging.DEBUG + 1) # default logger should not be used anyways!
__LOGGER = logging.getLogger('sigglitches')

we append the path so that we can use the chipwhisperer library

In [3]:
import sys
if '../../../software' not in sys.path:
    sys.path.append('../../../software')
if 'python-dilithium' not in sys.path:
    sys.path.append('python-dilithium')
sys.path

['/home/alexis/chipwhisperer.dilithium/jupyter/courses/fault101',
 '/usr/lib/python38.zip',
 '/usr/lib/python3.8',
 '/usr/lib/python3.8/lib-dynload',
 '',
 '/home/alexis/.venvs/chipwhisperer.dilithium/lib/python3.8/site-packages',
 '../../../software',
 'python-dilithium']

In [4]:
import chipwhisperer as cw
import importlib
import json
import uuid
import numpy as np
from chipwhisperer.capture.targets import TargetIOError, TargetTimeoutError
from dilithium import Dilithium
d = Dilithium(2)
cw, d

(<module 'chipwhisperer' from '/home/alexis/chipwhisperer.dilithium/jupyter/courses/fault101/../../../software/chipwhisperer/__init__.py'>,
 <dilithium.Dilithium at 0x7f5d704389d0>)

In [5]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-dilithium-ref
make PLATFORM=$1 CRYPTO_TARGET=NONE CRYPTO_OPTIONS=NONE SS_VER=$2

SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
make[1]: '.dep' is up to date.
SS_VER set to SS_VER_2_1
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc (15:9-2019-q4-0ubuntu1) 9.2.1 20191025 (release) [ARM/arm-9-branch revision 277599]
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling:
-en     simpleserial-dilithium-ref.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Compiling:
-en     .././simpleserial/simpleserial.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4_hal.c ...
mkdir -p "objdir-CW308_STM32F4"


In file included from .././hal/stm32f4/stm32f4_hal.c:3:
.././hal/stm32f4/stm32f4_hal_lowlevel.h:108: warning: "STM32F415xx" redefined
  108 | #define STM32F415xx
      | 
<command-line>: note: this is the location of the previous definition


-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4_hal_lowlevel.c ...
mkdir -p "objdir-CW308_STM32F4"


In file included from .././hal/stm32f4/stm32f4_hal_lowlevel.c:39:
.././hal/stm32f4/stm32f4_hal_lowlevel.h:108: warning: "STM32F415xx" redefined
  108 | #define STM32F415xx
      | 
<command-line>: note: this is the location of the previous definition


-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4_sysmem.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4xx_hal_rng.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Assembling: .././hal/stm32f4/stm32f4_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m4 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -fmessage-length=0 -ffunction-sections -DF_CPU=7372800 -Wa,-gstabs,-adhlns=objdir-CW308_STM32F4/stm32f4_startup.lst -I.././simpleserial/ -I.././hal -I.././hal/stm32f4 -I.././hal/stm32f4/CMSIS -I.././hal/stm32f4/CMSIS/core -I.././hal/stm32f4/CMSIS/device -I.././hal/stm32f4/Legacy -I.././crypto/ .././hal/stm32f4/stm32f4_startup.S -o objdir-CW308_STM32F4/stm32f4_startup.o
.
LINKING:
-en     simpleserial-dilithium-ref-CW308_STM32F4.elf ...
-e Done!
.
Creating load file for Flash: simpleserial-dilithium-ref-CW308_STM32F4.hex
arm-none-eabi-objcopy -O ihex -R .eeprom -R .fuse -R .lock -R .signature simpleserial-dilithium-ref-CW308_STM32F4.elf simples

In [6]:
%run "../../Setup_Scripts/Setup_Generic_Dilithium.ipynb"

(ChipWhisperer NAEUSB WARNING|File naeusb.py:713) Your firmware (0.30) is outdated - latest is 0.64 See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information


INFO: Found ChipWhisperer😍


In [7]:
fw_path = "../../../hardware/victims/firmware/simpleserial-dilithium-ref/simpleserial-dilithium-ref-{}.hex".format(PLATFORM)

In [8]:
#cw.program_target(scope, prog, fw_path)

In [9]:
# found using
# target.filter_msgs_one_iter([struct.pack('!H', i) for i in range(10)])
msgs_no_retry = [b'\x00\x01', b'\x00\x02', b'\x00\x07', b'\x00\t']
the_msg = msgs_no_retry[0]

In [10]:
def dilithium2_polyz_unpack_settings():
    scope.default_setup()
    scope.cglitch_setup() # default_setup for clock glitching
    scope.clock.adc_src = "clkgen_x1" # we do not need x4 as we do not capture traces; maybe speedup?
    scope.adc.samples = 0 # maybe speedup?
    scope.adc.timeout = .000000002 # wtf why is this not working

def get_duration():
    dilithium2_polyz_unpack_settings()
    target.reboot_flush()
    scope.arm()
    target.loop(TIMEOUT)
    timed_out = scope.capture()
    assert not timed_out
    return scope.adc.trig_count;

def get_sig_correct() -> bytes:
    dilithium2_polyz_unpack_settings()
    target.sign(the_msg, timeout=TIMEOUT)
    return target.get_sig()

dilithium2_polyz_unpack_settings()
duration = get_duration()
iter_duration = duration // NUM_ITERS
sig_correct = get_sig_correct()
_, z_correct, _ = d._unpack_sig(sig_correct)
poly_correct = z_correct[0]

print(f'clock cycle duration all = {duration}')
print(f'clock cycle duration iteration = {iter_duration}')
z_correct

clock cycle duration all = 4698
clock cycle duration iteration = 73


array([[-100321,   65070,    9592, ...,  103227,   -3425,   17889],
       [-107453,  -81618,   31730, ...,  122362,  -45288,   23573],
       [  88900, -128866, -104785, ...,  -48187, -104331,   71746],
       [  -9268,   50775,   -6038, ..., -118320,  130605,  -62888]])

In [11]:
def get_iter(poly_candidate: np.ndarray, coeffs_per_iter: int = 4, beta: int = BETA) -> bool:
    """
    0 = all coeffs zero; 1 - 63 = some non faulted in the beginning rest zero;
    64 = no fault; None = not our fault model (i.e. faults within iters)
    """
    for i in range(0, NUM_ITERS + 1):
        # index of the first zero coefficient due to a fault
        split_index = i * coeffs_per_iter
        if np.all(np.abs(poly_candidate[split_index:]) <= beta):
            return i
    return None

In [12]:
#glitch_spots = [4224, 4225, 4226, 2179, 2180, 2181, 2182, 2325, 2326, 2327, 2328, 2472, 2473, 2474, 4150, 4151, 3128, 3129, 3130, 1595, 1596, 1597, 1598, 3131, 4152, 4153, 3785, 3786, 3787, 3788, 1741, 1742, 1231, 1232, 1233, 1743, 1744, 2253, 2254, 2255, 4443, 4444, 4445, 2398, 2399, 2400, 2401, 354, 355, 356, 357, 4588, 4589, 4590, 4591, 4297, 4298, 4299, 4223]
glitch_spots = list(range(5 * iter_duration - 20, 5 * iter_duration + 20 + 1)) # target iter #5
width_start = 5
width_stop = 8
offset_start = 5
offset_stop = 8
repeat = 1
step = 1
redo = 1

In [13]:
# perfect 7
glitch_spots = list([360, 361, 361]) # target iter #5
width_start = 6.5
width_stop = 7.5
offset_start = 5.5
offset_stop = 6.5
repeat = 1
step = 1/3
redo = 10

(361, 5.859375, 7.03125)

(361, 5.859375, 7.03125)

In [14]:
# perfect all?
print(duration)
glitch_spots = range(duration) # target iter #5
width_start = 6.640625
width_stop = 6.640625
offset_start = 6.640625
offset_stop = 6.640625
repeat = 1
step = 1
redo = 3

(361, 6.640625, 6.640625)

4698


(361, 6.640625, 6.640625)

In [15]:
def width_sieve(widths: list) -> list:
    res = []
    for width in widths:
        scope.glitch.width = width
        res.append(scope.glitch.width)
    res = list(set(res))
    res.sort()
    return res
dat = np.arange(5, 6, 1/3).tolist()
print(dat)
print(width_sieve(dat))

[5.0, 5.333333333333333, 5.666666666666666]
[5.078125, 5.46875, 5.859375]


In [16]:
width_width = (width_stop - width_start + 1) / step
offset_width = (offset_stop - offset_start + 1) / step
ext_offset_width = len(glitch_spots)
print(width_width, offset_width, ext_offset_width)
normal_time = width_width * offset_width * ext_offset_width * redo
print(normal_time)
t = TIMEOUT * normal_time / 1000
print(f'Expected sig time: {t}s = {t/60}min = {t/3600}h')
t_all = t * 2.5 # what was that exactly again?
print(f'Expected sig + transfer time: {t_all}s = {t_all/60}min = {t_all/3600}h')

1.0 1.0 4698
14094.0
Expected sig time: 7187.94s = 119.79899999999999min = 1.9966499999999998h
Expected sig + transfer time: 17969.85s = 299.4975min = 4.991625h


In [17]:
import chipwhisperer.common.results.glitch as glitch
gc = glitch.GlitchController(groups=[f"perfect_{i}" for i in range(NUM_ITERS)] + ["success", "reset", "normal"], parameters=["width", "offset"])
gc.display_stats()

IntText(value=0, description='perfect_0 count:', disabled=True)

IntText(value=0, description='perfect_1 count:', disabled=True)

IntText(value=0, description='perfect_2 count:', disabled=True)

IntText(value=0, description='perfect_3 count:', disabled=True)

IntText(value=0, description='perfect_4 count:', disabled=True)

...

IntText(value=0, description='success count:', disabled=True)

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

FloatSlider(value=0.0, continuous_update=False, description='width setting:', disabled=True, max=10.0, readout…

FloatSlider(value=0.0, continuous_update=False, description='offset setting:', disabled=True, max=10.0, readou…

In [18]:
sigs_faulted = []
sigs_faulted_params = []

In [19]:
gc.set_range("offset", offset_start, offset_stop)
gc.set_range("width", width_start, width_stop)
gc.set_global_step(step)
scope.glitch.repeat = repeat
target.reboot_flush()

b'^A=3\x00\x0b\xcfx\x00@\x00\x00\x02@\x06\x80\x80P\x000\xc1\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x0bb\x07boot ok\xc1\x00'

In [20]:
def one_try(scope, target, gc, message, sigs_faulted: list, sigs_faulted_params: list) -> bool:
    scope.arm()
    try:
        target.sign(message, TIMEOUT)
        
        trigger_timeout = scope.capture()
        if trigger_timeout:
            gc.add("reset", (scope.glitch.width, scope.glitch.offset))
            __LOGGER.info('fail due to trigger timeout')
            target.reboot_flush()
            return
        
    except TargetTimeoutError as e:
        gc.add("reset", (scope.glitch.width, scope.glitch.offset))
        __LOGGER.info(f'failed due to read timeout: {e}')
        target.reboot_flush()
        return
    except TargetIOError as e:
        gc.add("reset", (scope.glitch.width, scope.glitch.offset))
        __LOGGER.info(f'failed due to TargetIOError error: {e}')
        target.reboot_flush()
        return
    else:
        try:
            sig_candidate_packed = target.get_sig()
            _, z_candidate, _ = d._unpack_sig(sig_candidate_packed)
            
            if np.all(z_candidate == z_correct):
                gc.add("normal", (scope.glitch.width, scope.glitch.offset))
                return
            i = get_iter(z_candidate[0])

            if i == NUM_ITERS:
                gc.add("success", (scope.glitch.width, scope.glitch.offset))
            elif 0 <= i < NUM_ITERS:
                gc.add(f"perfect_{i}", (scope.glitch.width, scope.glitch.offset))
            else:
                __LOGGER.warning('Unexpected iteration!')
            
            __LOGGER.info("Glitched! Ext offset: {} Offset: {} tWidth: {}".format(scope.glitch.ext_offset, scope.glitch.offset, scope.glitch.width))
            
            sigs_faulted.append(sig_candidate_packed)
            sigs_faulted_params.append((scope.glitch.ext_offset, scope.glitch.offset, scope.glitch.width))
        except TargetIOError as e:
            gc.add("reset", (scope.glitch.width, scope.glitch.offset))
            __LOGGER.info(f'Failed to read sig: {e}')
            target.reboot_flush()

In [21]:
start_time = time.time()
for glitch_setting in gc.glitch_values():
    scope.glitch.offset = glitch_setting[1]
    scope.glitch.width = glitch_setting[0]
    for i in glitch_spots:
        scope.glitch.ext_offset = i
        for j in range(redo):
            one_try(scope, target, gc, the_msg, sigs_faulted, sigs_faulted_params)
end_time = time.time()
total_duration = end_time - start_time
print(f'total duration: {total_duration}s {total_duration/60}min {total_duration/3600}h')

(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly t

(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly t

(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
INFO:sigglitches:Glitched! Ext offset: 225 Offset: 6.640625 tWidth: 6.640625
(ChipWhisperer Target WARNING|File SimpleSerial2.py:508) Unexpected frame byte in CWbytearray(b'84 72 80 91 80 ab 62 ea 13 15 f1 40 ea ee 08 93 88 04 f8 14 56 94 50 e6 17 16 60 25 a8 30 3b b2 be 36 9d 50 89 77 5d 99 95 f8 cb 46 6d c0 fe 66 cd 6e ae 34 08 33 a7 90 06 23 6a 33 10 e4 92 f6 54 74 27 53 94 63 b8 bf 28 d3 d9 8a 92 28 f8 71 8e 47 2d 2e 93 05 f2 25 4f 22 62 0f 74 15 73 cd 3d 31 17 4b 02 70 b3 5e b4 eb 4e b9 3f 4e 96 40 ea 99 6f f5 23 3d 63 77 0d 8e 65 cd 3a 06 6d e3 47 34 9b 00 03')
INFO:SimpleSerial2Dilithium:got an BlockingIOError exception: target logger logged a warning during simpleserial_read: ["Unexpected frame byte in CWbytearray(b'84 72 80 91 80 ab 62 ea 13 15 f1 40 ea ee 08 93 88 04 f8 14 56 94 50 e6 17 16 60 25 a8 30 3b 

INFO:SimpleSerial2Dilithium:got an BlockingIOError exception: target logger logged a warning during simpleserial_read: ["Unexpected frame byte in CWbytearray(b'84 72 80 17 2d af 30 06 2b a9 79 47 68 03 76 c1 8a 61 0b 5f 53 af 76 bd f1 90 da 49 8c c1 89 27 59 0a a7 5e d8 f3 1d 99 56 4e f7 76 27 36 bd 3d dc c2 cf 58 b2 f6 25 c4 97 8f 7e a2 3f 75 0a 26 25 56 f7 fd 73 f6 62 c2 a4 aa 12 b1 3e e2 da 19 f5 83 86 9b 8d 32 b9 7f 84 9e d0 d3 20 d5 da 41 af 81 2a 92 54 47 a6 17 94 a7 4e a2 0d 88 4f 11 15 d1 f6 a2 a1 47 d8 bb c0 25 f4 d9 92 e0 16 9e 41 bc 00 03 65')"]; trying again ...
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 

(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly t

...

(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
INFO:sigglitches:Glitched! Ext offset: 3895 Offset: 6.640625 tWidth: 6.640625
INFO:sigglitches:Glitched! Ext offset: 3895 Offset: 6.640

(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly t

INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File S

(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:528) Didn't get all data 2, 4
(ChipWhisperer Target WARNING|File SimpleSerial2.py:5

INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:sigglitches:failed due to read timeout: Target cleanly timed out while generating a signature
(ChipWhisperer Target WARNING|File S

In [27]:
from collections import defaultdict
rees = defaultdict(lambda: 0)
print(len(sigs_faulted))
for faultnum, sig_faulted in enumerate(sigs_faulted):
    _, z_faulted, _ = d._unpack_sig(sig_faulted)
    if get_iter(z_faulted[0]) == 5:
        print(sigs_faulted_params[faultnum])
        rees[sigs_faulted_params[faultnum]] += 1
        
        for i in range(len(z_faulted)):
            diff = np.abs(np.abs(z_faulted[i]) - np.abs(z_correct[i]))
            if np.any(np.abs(np.abs(z_faulted[i]) - np.abs(z_correct[i])) > 2 * BETA):
                print(f'fault no {faultnum}, poly {i} has a fault! See diff below and pay attention to large values.')
                print(diff)
                print(z_faulted[i])
                print(sigs_faulted_params[faultnum])
rees

285
(299, 6.640625, 6.640625)
fault no 15, poly 0 has a fault! See diff below and pay attention to large values.
[     2     16      3     15     15      2      0      3      9     11
      0      4      6      5      4      0      4     13     13     14
   2292 130528   4039  41646  84673   8408  74820 109615  91395  91565
 115889  23795  38573  97374  68333  13673  29253  93749  25493  28447
  98989  35424 129180  35960  98767   5205 121648  36128  43037 126575
  51087  26818  88119  48845   6478  91649  26811 122939  26554  69252
  24713  59483  95941  40121  14566  50398 112980  68730  66385  78213
  86555  76860  64684  31292  67048 120553  94142  16128  41261 107475
 123515  92912  46059 106962 123703  54063  68047  27768  95539  34303
  88601  16753 106495 110907  79356 111892  61994  37950  98496  55488
  39177  40938  32602  76256  51229  41900  65423  16207  55752  94118
  50224 120085 128687  94385  61476  19566  88961  76797   2807  91201
  56332 118793  96827  15032 128022

defaultdict(<function __main__.<lambda>()>, {(299, 6.640625, 6.640625): 1})

In [29]:
result_data = {
    "sig_correct": list(sig_correct),
    "glitch_spots": list(glitch_spots),
    "width_start": width_start,
    "width_stop": width_stop,
    "offset_start": offset_start,
    "offset_stop": offset_stop,
    "repeat": repeat,
    "redo": redo,
    "repeat": repeat,
    "step": step,
    "sigs_faulted": list(map(lambda s: list(s), sigs_faulted)),
    "sigs_faulted_param_descripton": "0 = ext_offset, 1 = offset, 2 = width",
    "sigs_faulted_params": sigs_faulted_params
}
with open(f'allperfectFIXED{uuid.uuid4()}.json', 'w') as f:
    json.dump(result_data, f)
#json.dumps(result_data)

In [ ]:
%matplotlib notebook
d = {"success":None, "reset":None, "normal":None}
for i in range(NUM_ITERS):
    d[f'perfect_{i}'] = "go"
gc.results.plot_2d(plotdots=d)

In [28]:
scope.dis()
target.dis()